# 安徽安庆市项目
## 本地排放清单物种分配
## `Allocate Species in LEAQ`

---
*@author: Evan*\
*@date: 2023-10-05*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../../src/')
from namelist import *

In [2]:
bio = xr.open_dataset(local_bio_file)
bol = xr.open_dataset(local_bol_file)
bld = xr.open_dataset(local_bld_file)
dst = xr.open_dataset(local_dst_file)
elc = xr.open_dataset(local_elc_file)
rod = xr.open_dataset(local_rod_file)
nro = xr.open_dataset(local_nro_file)
pet = xr.open_dataset(local_pet_file)
sol = xr.open_dataset(local_sol_file)
oth = xr.open_dataset(local_oth_file)

sections = ['bio','bol','bld','dst','elc','rod','nro','pet','sol','oth']
section_dict = {section: globals()[section] for section in sections}

section_dict['rod']

<xarray.Dataset>
Dimensions:    (y: 138, x: 135)
Coordinates:
    longitude  (y, x) float32 ...
    latitude   (y, x) float32 ...
Dimensions without coordinates: y, x
Data variables:
    SO2        (y, x) float32 ...
    NOx        (y, x) float32 ...
    CO         (y, x) float32 ...
    PM10       (y, x) float32 ...
    PM25       (y, x) float32 ...
    VOCs       (y, x) float32 ...
    NH3        (y, x) float32 ...
    BC         (y, x) float32 ...
    OC         (y, x) float32 ...

In [3]:
coordinates = ['longitude','latitude']
local_species = [x for x in list(rod.variables.keys()) if x not in coordinates]
print(local_species)

meic_example = xr.open_dataset(meic_ind_file)
to_remove = ['TFLAG']
meic_species = [x for x in list(meic_example.variables.keys()) if x not in to_remove]
print(meic_species)

['SO2', 'NOx', 'CO', 'PM10', 'PM25', 'VOCs', 'NH3', 'BC', 'OC']
['NO2', 'NO', 'HONO', 'CO', 'SO2', 'SULF', 'NH3', 'ALK1', 'ALK2', 'ALK3', 'ALK4', 'ALK5', 'ETHE', 'PRPE', 'OLE1', 'OLE2', 'BDE13', 'ISOP', 'APIN', 'TERP', 'SESQ', 'BENZ', 'TOLU', 'ARO1', 'OXYL', 'MXYL', 'PXYL', 'B124', 'ARO2MN', 'ACYE', 'HCHO', 'CCHO', 'RCHO', 'BALD', 'ACET', 'MEK', 'PRD2', 'MEOH', 'ETOH', 'FACD', 'AACD', 'PACD', 'GLY', 'MGLY', 'BACL', 'CRES', 'ACRO', 'MACR', 'MVK', 'IPRD', 'RNO3', 'NAPH', 'SOAALK', 'PSO4', 'PNO3', 'PCL', 'PNH4', 'PNA', 'PMG', 'PK', 'PCA', 'POC', 'PNCOM', 'PEC', 'PFE', 'PAL', 'PSI', 'PTI', 'PMN', 'PH2O', 'PMOTHR', 'PMC', 'HCL', 'CL2']


In [4]:
def multiply_with_factor(data_dict, factor):
    outdict = {}
    for key, value in data_dict.items():
        if isinstance(value, np.ndarray) and value.ndim == 2:
            new_data = factor[:, np.newaxis, np.newaxis, np.newaxis] * value
            outdict[key] = new_data
    
    return outdict

In [5]:
# 人为源排放后续使用刘老师的代码运行，变量需要严格遵守以下排序
order_sequence = ['NO2','NO','HONO','CO','SO2','SULF','HCHO',
                  'MEOH','AACD','PACD','RNO3','ACET','CRES',
                  'CCHO','RCHO','MEK','FACD','PRD2','MGLY',
                  'IPRD','GLY','BACL','BALD','MACR','MVK',
                  'ACRO','ETHE','PRPE','BDE13','ISOP','APIN',
                  'ACYE','BENZ','TOLU','MXYL','OXYL','PXYL',
                  'B124','ETOH','ALK1','ALK2','ALK3','ALK4',
                  'ALK5','SOAALK','OLE1','OLE2','ARO1','ARO2MN',
                  'NAPH','TERP','SESQ','CL2','HCL',
                  'PSO4','PNO3','PCL','PNH4','PNA','PMG','PK',
                  'PCA','POC','PNCOM','PEC','PFE','PAL','PSI',
                  'PTI','PMN','PH2O','PMOTHR','PMC','NH3']

In [6]:
pm_xls = pd.read_excel(datadir+'VOC_PM_species.xlsx',sheet_name='PM25')
voc_xls = pd.read_excel(datadir+'VOC_PM_species.xlsx',sheet_name='VOC')
temporal_xls = pd.read_csv(datadir+'monthly_temporal.csv')

for section in sections:
    print(f'Processing {section}')
    ds = section_dict[section]
    monthly = temporal_xls[section].values
    
    #! general compounds
    SULF = ds.SO2.values*0.05/98
    SO2  = ds.SO2.values/64

    NO   = ds.NOx.values*0.9/30
    NO2  = ds.NOx.values*0.1/46
    HONO = ds.NOx.values*0/47

    CO = ds.CO.values/28

    NH3 = ds.NH3.values/17

    PEC = ds.BC.values
    POC = ds.OC.values
    
    CL2 = np.zeros_like(NO)
    HCL = np.zeros_like(NO)
    
    general_list = ['SULF','SO2','NO','NO2','HONO','CO','NH3','PEC','POC','CL2','HCL']
    general_dict = {name: globals()[name] for name in general_list}

    #! PM species
    pm_list    = pm_xls['species'][2:].values
    if section != 'agr':
        pm_factors = pm_xls[section][2:].values
    else:
        pm_factors = np.zeros_like(pm_list)
    pm_dict    = {pm: ds.PM25.values * factor for pm,factor in zip(pm_list,pm_factors)}
    
    pm_dict['PMC'] = ds.PM10.values - ds.PM25.values
    
    #! VOC species
    voc_list    = voc_xls['species'].values
    voc_factors = voc_xls['ratio'].values
    voc_dict    = {voc: ds.VOCs.values * factor for voc,factor in zip(voc_list,voc_factors)}
    # ALK3
    voc_dict['ETOH'] = voc_dict['ALK3'] * 0.3708
    voc_dict['ALK3'] = voc_dict['ALK3'] * 0.6292
    # ALK4 ALK5
    voc_dict['SOAALK'] = voc_dict['ALK4'] * 0.1 + voc_dict['ALK5'] * 0.7
    voc_dict['ALK4']   = voc_dict['ALK4'] * 0.9
    voc_dict['ALK5']   = voc_dict['ALK5'] * 0.3
    # OLE1
    voc_dict['PRPE'] = voc_dict['OLE1'] * 0.4952
    voc_dict['OLE1'] = voc_dict['OLE1'] * 0.5048
    # OLE2
    voc_dict['BDE13'] = voc_dict['OLE2'] * 0.0762
    voc_dict['OLE2']  = voc_dict['OLE2'] * 0.9238
    # TERP
    voc_dict['APIN'] = voc_dict['TERP'] * 0.2222
    voc_dict['SESQ'] = voc_dict['TERP'] * 0.2222
    voc_dict['TERP'] = voc_dict['TERP'] * 0.5556
    # ARO1
    voc_dict['TOLU'] = voc_dict['ARO1'] * 0.7222
    voc_dict['ARO1'] = voc_dict['ARO1'] * 0.2778
    # ARO2
    voc_dict['OXYL']   = voc_dict['ARO2MN'] * 0.1271
    voc_dict['MXYL']   = voc_dict['ARO2MN'] * 0.2034
    voc_dict['PXYL']   = voc_dict['ARO2MN'] * 0.1610
    voc_dict['B124']   = voc_dict['ARO2MN'] * 0.0593
    voc_dict['NAPH']   = voc_dict['ARO2MN'] * 0.0180
    voc_dict['ARO2MN'] = voc_dict['ARO2MN'] * 0.4312
    # MACR
    voc_dict['ACRO'] = voc_dict['MACR'] * 0.6667
    voc_dict['MACR'] = voc_dict['MACR'] * 0.3333
    
    #! merge
    merged_dict = {**general_dict,**pm_dict,**voc_dict}
    print('2d-variables Dict Completed')
    
    #! broadcasting for monthly allocate
    monthdata = multiply_with_factor(merged_dict,monthly)
    print('Monthly Allocation Completed')
    
    #! write into dataset
    dims = ['TSTEP','LAY','ROW','COL']
    dataset=None
    dataset = xr.Dataset(
        data_vars = {var_name: (dims,data) for var_name, data in monthdata.items()},
        coords = dict(
            TSTEP = np.arange(0,12),
            LAY = np.arange(0,1),
            ROW = np.arange(0,138),
            COL = np.arange(0,135)
        )
    )
    #! 变量重排序
    dataset = dataset[order_sequence]
    dataset.to_netcdf(allocated_dir + f'emis.CN3AH_135X138.{section}.ncf')
    

Processing bio
2d-variables Dict Completed
Monthly Allocation Completed
Processing bol
2d-variables Dict Completed
Monthly Allocation Completed
Processing bld
2d-variables Dict Completed
Monthly Allocation Completed
Processing dst
2d-variables Dict Completed
Monthly Allocation Completed
Processing elc
2d-variables Dict Completed
Monthly Allocation Completed
Processing rod
2d-variables Dict Completed
Monthly Allocation Completed
Processing nro
2d-variables Dict Completed
Monthly Allocation Completed
Processing pet
2d-variables Dict Completed
Monthly Allocation Completed
Processing sol
2d-variables Dict Completed
Monthly Allocation Completed
Processing oth
2d-variables Dict Completed
Monthly Allocation Completed
